In [10]:
from utils import *
import matplotlib.pyplot as plt
import librosa
import torch
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Model

import torch
from imagen_pytorch import Unet3D, ElucidatedImagen, ImagenTrainer
from utils import gif75speaker
import numpy as np
from torchvision import transforms

In [ ]:
# traindata = gif75speaker(transform = data_transforms['val'])
# (img, aud, preceding) = traindata[20000]
# plt.imshow(img[:,0,:].permute(1, 2, 0))

In [11]:
unet1 = Unet3D(dim = 64, dim_mults = (1, 2, 4, 8)).cuda()

unet2 = Unet3D(dim = 64, dim_mults = (1, 2, 4, 8)).cuda()

# elucidated imagen, which contains the unets above (base unet and super resoluting ones)

imagen = ElucidatedImagen(
    text_embed_dim=1024,
    unets = (unet1, unet2),
    image_sizes = (64, 64),
    random_crop_sizes = (None, 16),
    temporal_downsample_factor = (1, 1),        # in this example, the first unet would receive the video temporally downsampled by 2x
    num_sample_steps = 10,
    cond_drop_prob = 0.1,
    sigma_min = 0.002,                          # min noise level
    sigma_max = (80, 160),                      # max noise level, double the max noise level for upsampler
    sigma_data = 0.5,                           # standard deviation of data distribution
    rho = 7,                                    # controls the sampling schedule
    P_mean = -1.2,                              # mean of log-normal distribution from which noise is drawn for training
    P_std = 1.2,                                # standard deviation of log-normal distribution from which noise is drawn for training
    S_churn = 80,                               # parameters for stochastic sampling - depends on dataset, Table 5 in apper
    S_tmin = 0.05,
    S_tmax = 50,
    S_noise = 1.003,
).cuda()


imagen.load_state_dict(torch.load('/mnt/c/Users/PCM/Documents/GitHub/SPAN-rtmri/checkpoints/imagen-video-audio60pho-notIgnoreTime-10frames-206'))
imagen.eval()

ElucidatedImagen(
  (lowres_noise_schedule): GaussianDiffusionContinuousTimes()
  (unets): ModuleList(
    (0): Unet3D(
      (init_conv): CrossEmbedLayer(
        (convs): ModuleList(
          (0): Conv3d(3, 32, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
          (1): Conv3d(3, 16, kernel_size=(1, 7, 7), stride=(1, 1, 1), padding=(0, 3, 3))
          (2): Conv3d(3, 16, kernel_size=(1, 15, 15), stride=(1, 1, 1), padding=(0, 7, 7))
        )
      )
      (to_time_hiddens): Sequential(
        (0): LearnedSinusoidalPosEmb()
        (1): Linear(in_features=17, out_features=256, bias=True)
        (2): SiLU()
      )
      (to_time_cond): Sequential(
        (0): Linear(in_features=256, out_features=256, bias=True)
      )
      (to_time_tokens): Sequential(
        (0): Linear(in_features=256, out_features=128, bias=True)
        (1): Rearrange('b (r d) -> b r d', r=2)
      )
      (norm_cond): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (text_to_cond):

In [ ]:
# aud_embs = torch.load(f'./datasets/audios/sub070_2drt_12_picture1_video.pt')
# aud_embs.shape

In [12]:
def add_noise_video(sample_img, image_sizes = (64, 64), timesteps=1000, times = 50):
    ret = []
    for i in range(10):
        x_noisy, _, _, _ = add_noise(sample_img[:,:,i,:].cpu(), image_sizes = image_sizes, timesteps=timesteps, times = times)
        ret.append(x_noisy)
    return torch.stack(ret, axis=2)
# aaa = add_noise_video(sample_img)

### Visualization for Audio-2-Images

In [ ]:

listimgs = []
!rm -rf vidgen_log
!mkdir vidgen_log

sample_img = imagen.sample(text_embeds = aud.cuda(), video_frames = 20, stop_at_unet_number=1, skip_steps=0)
init_images = add_noise_video(sample_img, image_sizes = (64, 64), timesteps=1000, times = 300).cuda()
for i in range(400,500,10):
    print(i)
    aud = aud_embs[:,i:i+10,:]
    sample_img = imagen.sample(text_embeds = aud.cuda(), video_frames = 20, stop_at_unet_number=1, skip_steps=0, init_images = init_images, cond_video_frames = sample_img.cuda())
    init_images = add_noise_video(sample_img, image_sizes = (64, 64), timesteps=1000, times = 300).cuda()
    listimgs.append(sample_img)
    # sample_img[0].save(f'./vidgen_log/imagen-video-sample-{i}.png')

In [ ]:
torch.mean(aud, axis=1).shape

### Visualization for Audio-2-Gifs

In [9]:
torch.mean(audio_z, axis=0).unsqueeze(0).shape

torch.Size([1, 1024])

In [13]:
# Training model to Condition on preceding video frames

#Load unseen images
traindata = gif75speaker(transform = data_transforms['val'])
(img, audio_z, preceding) = traindata[20000]

#Load unseen speech
aud_embs = torch.load(f'./datasets/audios-eng-pho/sub002_2drt_12_picture1_video.pt')

listimgs = []
!rm -rf vidgen_log
!mkdir vidgen_log

aud = aud_embs[:,390:390+10,:]# torch.mean(aud_embs[:,380:380+10,:], axis=1).unsqueeze(0) #aud_embs[:,380:380+10,:]#
# sample_img = imagen.sample(text_embeds = aud.cuda(), video_frames = 20, stop_at_unet_number=1, skip_steps=0, cond_video_frames=preceding.unsqueeze(0).cuda())
# init_images = add_noise_video(sample_img, image_sizes = (64, 64), timesteps=1000, times = 300).cuda()

for i in range(400,600,10):
    print(i)
    aud = torch.mean(aud_embs[:,i:i+10,:], axis=1).unsqueeze(0) #aud_embs[:,i:i+10,:]#
    sample_img = imagen.sample(text_embeds = aud.cuda(), video_frames = 10, stop_at_unet_number=1, skip_steps=0, cond_video_frames = preceding.unsqueeze(0).cuda())
    preceding = sample_img[0,:,-2:,:]
    # init_images = add_noise_video(sample_img, image_sizes = (64, 64), timesteps=1000, times = 300).cuda()
    listimgs.append(sample_img)
    # sample_img[0].save(f'./vidgen_log/imagen-video-sample-{i}.png')

400


0it [00:00, ?it/s]

sampling time step:   0%|          | 0/10 [00:00<?, ?it/s]

410


0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

sampling time step:   0%|          | 0/10 [00:00<?, ?it/s]

590


0it [00:00, ?it/s]

sampling time step:   0%|          | 0/10 [00:00<?, ?it/s]

In [14]:
a = torch.transpose(torch.stack(listimgs, axis=0).squeeze(1),1,2)
a.shape
# b = a.reshape(50,3,64,64)

torch.Size([20, 10, 3, 64, 64])

In [16]:
b = a.reshape(200,3,64,64)
imgs = [transforms.ToPILImage()(img) for img in b]
# duration is the number of milliseconds between frames; this is 40 frames per second
imgs[0].save(f'gif-sample-video.gif', save_all=True, append_images=imgs[1:], duration=20, loop=0)
# torch.save(imagen.state_dict(), f'imagen-video-test')

## Visualization side-by-side videos and grounftruth

In [15]:
# def load_frames(image: Image, mode='RGB'):
#     return np.array([
#         np.array(frame.convert(mode))
#         for frame in ImageSequence.Iterator(image)
#     ])

# with Image.open(glob.glob('./datasets/gifs/*')[20000]) as im:
#     imgs2_arr = load_frames(im)

imgs2 = []
gifs2 = glob.glob('./datasets/gifs/*')[20000:20020]
i = 0
for gif in gifs2:
    for frame in ImageSequence.Iterator(Image.open(gif)):
        # if(i % 2 == 0):
        imgs2.append(frame)
        i = i + 1

# imgs2 = [frames for frames in ImageSequence.Iterator(Image.open(glob.glob('./datasets/gifs/*')[20000]))]

In [16]:
images = imgs[:3]
widths, heights = zip(*(i.size for i in images))

total_width = sum(widths)
max_height = max(heights)

combine_gif = []

for i in range(200):
  new_im = Image.new('RGB', (128, 64))

  x_offset = 0
  # for im in images:
  new_im.paste(imgs[int(i)], (x_offset,0))
  x_offset += 64
  new_im.paste(imgs2[i], (x_offset,0))
  combine_gif.append(new_im)

  

# new_im
# new_im.save('test.jpg')

In [17]:
combine_gif[0].save(f'gif-sample-60pho-preceding-2:-sidebyside.gif', save_all=True, append_images=combine_gif[1:], duration=20, loop=0)

In [ ]:
# imgs2[0].save(f'gif-sample-groundtruth.gif', save_all=True, append_images=imgs[1:], duration=40, loop=0)